In [1]:
import os
import glob
import numpy as np
import time
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

In [19]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)
# Abrir el GSpreadsheet
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=1166684204#gid=1166684204")

In [20]:
data_input = 'input_sim' #nombre de la hoja de datos de entrada

sheet = spreadsheet.worksheet(data_input) # Abrir la hoja de cálculo con los datos de entrada
data = sheet.get_all_records()

#Ahora puedo manejar todos los datos con el DataFrame de pandas sin modificar la hoja de cálculo
df_simulations = pd.DataFrame(data)
df_simulations.head() # Verifico lo importado

,description,simulation_name,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,src_ang_start,src_ang_end,src_ang_step
0,"Variación de altura de la fuente, pasos chicos...",height_source_3m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,3,0,180,5
1,"Variación de altura de la fuente, pasos chicos...",height_source_4m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,4,0,180,5
2,"Variación de altura de la fuente, pasos chicos...",height_source_5m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,5,0,180,5
3,"Variación de altura de la fuente, pasos chicos...",height_source_6m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,6,0,180,5
4,"Variación de altura de la fuente, pasos chicos...",height_source_7m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,7,0,180,5


In [21]:
df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla
df_simulations.head() # Verificar el resultado

,description,simulation_name,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,src_ang_start,src_ang_end,src_ang_step,room_dim
0,"Variación de altura de la fuente, pasos chicos...",height_source_3m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,3,0,180,5,"(20, 40, 10)"
1,"Variación de altura de la fuente, pasos chicos...",height_source_4m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,4,0,180,5,"(20, 40, 10)"
2,"Variación de altura de la fuente, pasos chicos...",height_source_5m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,5,0,180,5,"(20, 40, 10)"
3,"Variación de altura de la fuente, pasos chicos...",height_source_6m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,6,0,180,5,"(20, 40, 10)"
4,"Variación de altura de la fuente, pasos chicos...",height_source_7m_very_short_rev,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,5,7,0,180,5,"(20, 40, 10)"


## Simulación de audios captados por micrófonos 

In [22]:
for idx, sim in df_simulations.iterrows(): # Itero cada simulación
    for angle in range(sim['src_ang_start'], sim['src_ang_end'] + 1, sim['src_ang_step']):
        src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
            wav_path=f'audios/anechoic/{sim["audio"]}',
            out_dir=f'audios/output/{sim['simulation_name']}/{angle}',
            audio_name=f'{angle}_{sim["simulation_name"]}',
            fs=48000,
            room_dim=sim["room_dim"],
            rt60=sim["rt60"],
            snr_db=sim["snr_db"],
            n_mics=sim["n_mics"],
            mic_d=sim["mic_d"],
            mic_z=sim["mic_z"],
            mic_directivity=sim["mic_directivity"],
            src_dist=sim["src_dist"],
            src_az_deg=angle,
            src_z=sim["src_z"],
            save_audio=True
        )

## Calcular DOA desde archivos WAV

In [23]:
methods = ['classic', 'phat', 'roth', 'scot']  # Método de estimación a utilizar
#inicar computo de cuanto demora el código con time.time()
for method in methods:
    for idx, sim in df_simulations.iterrows(): # Itero cada simulación
        print(f"Simulación {sim['simulation_name']} con método {method} cada {sim['src_ang_step']} grados") 
        start_time = time.time()
        # Cargar los archivos de audio generados
        simulations = os.listdir(f'audios/output/{sim["simulation_name"]}')
        # Creación del DataFrame para almacenar los resultados
        df_results = pd.DataFrame(np.repeat(df_simulations.iloc[idx:idx+1, 2:14].values, len(simulations), axis=0), columns=df_simulations.columns[2:14])

        row = 0  
        for angle in simulations:
            # Obtener las señales de cada posición simulada
            mic_signals = glob.glob(f'audios/output/{sim["simulation_name"]}/{angle}/*.wav')
            avg_angle, avg_tdoa, angles_per_mic_ref, tdoas_per_mic_ref = estimation.estimate_doa(
            signals=mic_signals,
            fs=48000,
            d=sim["mic_d"],
            method=method
            )
            
            df_results.at[row, 'method'] = method
            df_results.at[row, 'angle'] = int(angle)
            df_results.at[row, 'avg_angle'] = avg_angle
            df_results.at[row, 'angle_error'] = np.abs(avg_angle - int(angle))
            df_results.at[row, 'desv_std_angle'] = np.std([angle for angles in angles_per_mic_ref for angle in angles])
            
            row += 1
            
        # print(f"Tiempo de ejecución para la simulación {sim['simulation_name']} con ángulos de {sim['src_ang_start']} a {sim['src_ang_end']}: {time.time() - start_time:.2f} segundos") 
        # Ordenar valores de df
        sheet_name = f'{sim["simulation_name"]}_{method}'
        # df_results['angle'] = pd.to_numeric(df_results['angle'], errors='coerce')
        df_results = df_results.sort_values(by=['angle']).reset_index(drop=True)

        # Crear csv en una carpeta llamada csv_results
        if not os.path.exists('csv_results'):
            os.makedirs('csv_results')
            
        df_results.to_csv(f'csv_results/{sheet_name}.csv')
        # Subir a GSpreadsheet creando una nueva hoja
        
        # try:
        #     spreadsheet.add_worksheet(title=f'{sheet_name}', rows="100", cols="20")
        #     results_sheet = spreadsheet.worksheet(f'{sheet_name}')
        #     results_sheet.update([df_results.columns.values.tolist()] + df_results.values.tolist())
        # except Exception as e:
        #     print(f"Error al subir los resultados a Google Sheets: {e}, ver si no existe la hoja ya en el GSpreadsheet")
        
    
    #     # Guardar resultados en cada fila individual
        
    #     # df_results.head()
    #     row += 1
        # df_results.at[idx, 'angle_error'] = np.abs(avg_angle - int(angle))
        # df_results.at[idx, 'angle_error_pct'] = (df_results.at[idx, 'angle_error'] / angle) * 100 if sim["src_az_deg"] != 0 else 0
        # df_results.at[idx, 'std_angle'] = np.std([angle for angles in angles_per_mic_ref for angle in angles])

        # for i in range(len(angles_per_mic_ref)):
            # for j in range(len(angles_per_mic_ref[i])):
                # df_results.at[idx, f'angle_{i+1}{j+2}'] = angles_per_mic_ref[i][j]


Simulación height_source_3m_very_short_rev con método classic cada 5 grados
Simulación height_source_4m_very_short_rev con método classic cada 5 grados
Simulación height_source_5m_very_short_rev con método classic cada 5 grados
Simulación height_source_6m_very_short_rev con método classic cada 5 grados
Simulación height_source_7m_very_short_rev con método classic cada 5 grados
Simulación height_source_8m_very_short_rev con método classic cada 5 grados
Simulación height_source_9m_very_short_rev con método classic cada 5 grados
Simulación height_source_3m_very_short_rev con método phat cada 5 grados
Simulación height_source_4m_very_short_rev con método phat cada 5 grados
Simulación height_source_5m_very_short_rev con método phat cada 5 grados
Simulación height_source_6m_very_short_rev con método phat cada 5 grados
Simulación height_source_7m_very_short_rev con método phat cada 5 grados
Simulación height_source_8m_very_short_rev con método phat cada 5 grados
Simulación height_source_9m_ve

In [ ]:
df_results.head(40)

,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,mic_z,mic_directivity,src_dist,src_z,method,angle,avg_angle,angle_error,desv_std_angle
0,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,0.0,18.899750,18.899750,3.822978
1,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,5.0,18.899750,13.899750,3.822978
2,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,10.0,19.483978,9.483978,2.938177
3,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,15.0,21.277105,6.277105,1.508941
4,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,20.0,29.138851,9.138851,2.438538
5,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,25.0,30.655757,5.655757,1.030420
6,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,30.0,36.707888,6.707888,1.967593
7,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,35.0,41.286908,6.286908,2.926508
8,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,40.0,43.835953,3.835953,1.213699
9,sine_sweep_24bit.wav,20,40,10,0.5,90,4,0.1,1.2,omni,9.5,1.2,scot,45.0,48.809462,3.809462,1.552247
